### 곡 제목에 생일이 들어가는 데이터 수집

https://www.melon.com/search/song/index.htm?q=%EC%83%9D%EC%9D%BC&section=&searchGnbYn=Y&kkoSpl=N&kkoDpType=#params%5Bq%5D=%25EC%2583%259D%25EC%259D%25BC&params%5Bsort%5D=hit&params%5Bsection%5D=all&params%5BsectionId%5D=&params%5BgenreDir%5D=&po=pageObj&startIndex=101

In [1]:
import pandas as pd
import re
import time
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

In [56]:
25*50+1

1251

In [98]:
headers ={'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
          'referer':'https://www.melon.com/search/song/index.htm?q=%EC%83%9D%EC%9D%BC&section=&searchGnbYn=Y&kkoSpl=N&kkoDpType='}


all_list = []
for index in range(26):
    startIndex = index * 50 + 1

    url = f"https://www.melon.com/search/song/index.htm?startIndex={startIndex}&pageSize=50&q=%25EC%2583%259D%25EC%259D%25BC&sort=hit&section=all&sectionId=&genreDir="
    res = requests.get(url, headers = headers)

    # 성공일 경우
    if res.status_code == 200:
        text = res.text

        bs = BeautifulSoup(res.text, 'html.parser')

        # rank 순위


        tr_lst = bs.select("tbody > tr")
        for tr in tr_lst:
            # print(tr)
            tds = tr.select("td")

            num = tds[1].text
            ellipsis = tds[2].select_one(".ellipsis")
            a_lst = ellipsis.select("a")
            song_id = a_lst[0]['href']
            d = re.compile("[\d]+")
            song_id = d.findall(song_id)[1]

            song_name = a_lst[0].text.replace(" 상세정보 페이지 이동", "")
            artist = tds[3].select_one("span").text


            album_id = tds[4].select_one('a')['href']
            d = re.compile("[\d]+")
            album_id = d.findall(album_id)[1]

            one_list = [num, song_name, artist, album_id, song_id]
            all_list.append(one_list)




        time.sleep(0.5)
    else:
        print("======= 에러 코드", res.status_code)
        break






In [100]:
all_list[:5]

[['1', '생일 축하곡 (Acid House Remix)', '터보', '348449', '1606440'],
 ['2', '생일 축하합니다', '뽀로로', '11170032', '36123520'],
 ['3', '너의 생일 (One Day)', '태연 (TAEYEON)', '10176822', '31133729'],
 ['4', '생일 축하 송', 'Various Artists', '101503', '2010111'],
 ['5', '생일 축하합니다 (생일축하송)', '베베팝', '10950002', '35079221']]

In [103]:
columns = ['num', 'song_name', 'artist', 'album_id', 'song_id']

result_df = pd.DataFrame(all_list, columns=columns)

In [105]:
result_df

,num,song_name,artist,album_id,song_id
0,1,생일 축하곡 (Acid House Remix),터보,348449,1606440
1,2,생일 축하합니다,뽀로로,11170032,36123520
2,3,너의 생일 (One Day),태연 (TAEYEON),10176822,31133729
3,4,생일 축하 송,Various Artists,101503,2010111
4,5,생일 축하합니다 (생일축하송),베베팝,10950002,35079221
...,...,...,...,...,...
1267,1268,생일 축하 노래,Various Artists,357041,1718761
1268,1269,생일 노래,Various Artists,357068,1719514
1269,1270,생일 축하 노래,Various Artists,30664,444264
1270,1271,생일 축하 노래,Various Artists,357059,1719280


In [108]:
result_df.to_excel('song_name.xlsx', index=False)

In [109]:
pd.read_excel('song_name.xlsx')

,num,song_name,artist,album_id,song_id
0,1,생일 축하곡 (Acid House Remix),터보,348449,1606440
1,2,생일 축하합니다,뽀로로,11170032,36123520
2,3,너의 생일 (One Day),태연 (TAEYEON),10176822,31133729
3,4,생일 축하 송,Various Artists,101503,2010111
4,5,생일 축하합니다 (생일축하송),베베팝,10950002,35079221
...,...,...,...,...,...
1267,1268,생일 축하 노래,Various Artists,357041,1718761
1268,1269,생일 노래,Various Artists,357068,1719514
1269,1270,생일 축하 노래,Various Artists,30664,444264
1270,1271,생일 축하 노래,Various Artists,357059,1719280


### song_id를 이용해 가사 수집

In [2]:
!pip install fake_useragent

In [3]:
from fake_useragent import UserAgent
from tqdm import tqdm

In [4]:
song_df = pd.read_excel('song_name.xlsx')

In [111]:
song_df.tail(3)

,num,song_name,artist,album_id,song_id
1269,1270,생일 축하 노래,Various Artists,30664,444264
1270,1271,생일 축하 노래,Various Artists,357059,1719280
1271,1272,생일 축하,Various Artists,8226,184564


In [5]:
# 텍스트에 있는 \n과 \t를 제거하고 문장 양옆의 띄어쓰기를 지운다.
def clean_text(text):
    text = re.sub('\\t', '', text)
    text = re.sub('\\n', '', text)
    text = text.strip()
    return text

In [6]:
ua = UserAgent().random


# {'songId':'가사'} 형식으로 dict를 만드는 함수
def make_lyrics_dict(songIdList):
    result_list = {}
    headers ={'User-Agent' : ua}

    for songId in tqdm(songIdList):
        url = f"https://www.melon.com/song/detail.htm?songId={songId}"
        res = requests.get(url, headers = headers)

        # 성공일 경우
        if res.status_code == 200:
            text = res.text

            bs = BeautifulSoup(res.text, 'html.parser')
            # lyrics 가사
            # song_name 가수명
            # artist 아티스트명
            lyrics = bs.select_one("#d_video_summary")


            # lyrics가 없을 경우 continue
            if lyrics is None:
                continue

            lyrics = bs.select_one("#d_video_summary").text
            lyrics = clean_text(lyrics)

            genre = bs.select(".wrap_info dd")[2].text.strip()

            result_list[songId] = [lyrics, genre]

            time.sleep(0.6)
        else:
            print("에러!! res.status_code:", res.status_code, "songId:", songId)
            break


    return result_list

In [7]:
songIdList = list(song_df['song_id'])
print('org_len:', len(songIdList))
songIdList = list(set(songIdList))
print('set_len:', len(songIdList))

org_len: 1272
set_len: 1272


In [122]:
lyrics_dict_300 = make_lyrics_dict(songIdList[:300])

100%|██████████| 300/300 [05:24<00:00,  1.08s/it]


In [123]:
lyrics_dict_600 = make_lyrics_dict(songIdList[300:600])

100%|██████████| 300/300 [05:30<00:00,  1.10s/it]


In [124]:
lyrics_dict_900 = make_lyrics_dict(songIdList[600:900])

100%|██████████| 300/300 [05:24<00:00,  1.08s/it]


In [8]:
lyrics_dict_1272 = make_lyrics_dict(songIdList[900:])

100%|██████████| 372/372 [08:47<00:00,  1.42s/it]


In [10]:
def save_text_file(filename, dic):
    # 텍스트 파일로 저장하기
    file = open(f"{filename}.txt", "w")
    file.write(str(dic))
    file.close()

In [11]:
save_text_file('lyrics_dict_1272', lyrics_dict_1272)

In [13]:
def change_dict(filename):
  f = open(filename, 'r')     # mode = 부분은 생략해도 됨
  str = f.read()

  return eval(str)

In [14]:
dict1 = change_dict('lyrics_dict_300.txt')
dict2 = change_dict('lyrics_dict_600.txt')
dict3 = change_dict('lyrics_dict_900.txt')
dict4 = change_dict('lyrics_dict_1272.txt')

In [15]:
len(dict1)

87

In [16]:
dict1.update(dict2)
dict1.update(dict3)
dict1.update(dict4)

In [17]:
len(dict1) # 반주곡도 많아서 적은 것 같다.

365

In [21]:
lyrics_dict = dict1

In [23]:
# CSV 파일 경로
csv_filename = "birthday_song_lyrics.csv"

# CSV 파일로 데이터 저장
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['Song_ID', 'Genre', 'Lyrics']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # CSV 파일의 헤더 작성
    writer.writeheader()

    # 딕셔너리 데이터를 CSV 파일에 작성
    for song_id, info in lyrics_dict.items():
        writer.writerow({'Song_ID': song_id, 'Genre': info[1], 'Lyrics': info[0]})

In [24]:
pd.read_csv("birthday_song_lyrics.csv")

,Song_ID,Genre,Lyrics
0,57347,발라드,이른 아침 눈을 떠 보니 오늘이 바로 그 날 네 곁에서 축복해주고 싶었지만 난 전화...
1,30400521,"키즈, 창작동요",루피 에디 슈퍼마켓 갔죠패티는 집을 꾸몄죠 포비는 고기를 구웠죠 뽀로로 심심했죠 ...
2,31645744,"키즈, 창작동요",생일 축하 합니다. 생일 축하 합니다. 당신의 생일을 축하합니다.생일 축하 합니다....
3,33251391,"인디음악, 포크/블루스",세상에 나오느라 수고했어 정말내 옆에 있어줘서 참 고마워잊을만하면 돌아오는내가 제일...
4,33136789,"키즈, 창작동요",친구야 생일 축하해 정말로 생일 축하해오늘은 너무나 기쁜날 우리 함께생일 축하 노래...
...,...,...,...
360,34701192,발라드,정신 없이 달려와시간이 얼마나 흘렀는지 모른 채다름 없는 날들과 같이그대 생일이 흘...
361,30572441,"CCM, 국내CCM",예쁜 친구야 생일 축하해멋진 친구야 생일 축하해언제까지나 밤하늘의 별처럼반짝 반짝 ...
362,33980315,"키즈, 영어동요",Happy birthday to youHappy birthday to youHapp...
363,1204189,발라드,온 동네 떠나갈 듯울어 젖히는 소리내가 세상에 첫선을 보이던바로 그 날이란다두리 둥...
